In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv('D:/download_D/digit-recognizer/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data=np.array(data)
m,n=data.shape
np.random.shuffle(data)
data_dev = data[0:1000].T
X_dev=data_dev[1:n]
Y_dev=data_dev[0]

data_train=data[1000:10000]
X=data_train[:,1:]/255
Y=data_train[:,0]
print(X.shape)

(9000, 784)


In [5]:
W1=(np.random.rand(785,100)-0.5)
W2=(np.random.rand(101,100)-0.5)
W3=(np.random.rand(101,10)-0.5)
print(W1.shape)

(785, 100)


In [6]:
def sigmoid(z):
    y=1/(1+np.exp(-z))
    return y

In [7]:
def addBias(x):
    bs=np.ones((len(x),1))
    return np.concatenate((bs,x),axis=1)

In [8]:
def forward_prop(W1,W2,W3,X):
    X=addBias(X)#(9000,785)
    Z1=np.matmul(X,W1)#(9000,785)x(785,100)
    A1=sigmoid(Z1)
    A1=addBias(A1)#(9000,101)
    Z2=np.matmul(A1,W2)#(9000,101)x(101,100)
    A2=sigmoid(Z2)
    A2=addBias(A2)#(9000,101)
    Z3=np.matmul(A2,W3)#(9000,101)x(101,10)
    A3=sigmoid(Z3)
    return A1,A2,A3,X

In [9]:
def backward_prop(A1,A2,A3,W2,W3,Y,Xi,e):
    dE=(A3-Y)#(9000,10)

    dE_dW3=np.matmul(A2.transpose(),dE)#(101,9000)x(9000,10)
    #
    dE_dA2=np.matmul(dE,W3.transpose())#(9000,10)x(10,101)
    dE_dZ2=dE_dA2*A2*(1-A2)#(9000,101)
    dE_dW2=np.matmul(A1.transpose(),dE_dZ2)#(101,9000)x(9000,101)
    #
    dE_dA1=np.matmul(dE_dZ2[:,1:],W2.transpose())#(9000,100)x(100,101)
    dE_dZ1=dE_dA1*A1*(1-A1)#(9000,101)
    dE_dW1=np.matmul(Xi.transpose(),dE_dZ1)#(785,9000)x(9000,101)
    #
    dW1=e*dE_dW1[:,1:]
    dW2=e*dE_dW2[:,1:]
    dW3=e*dE_dW3
    # print(Y-A3)
    return dW1,dW2,dW3,(Y-A3)**2


In [10]:
def makeLabel(num,num_class):
    vec = np.zeros(num_class)
    vec[num] = 1
    return vec
T = map(makeLabel,Y,[10]*len(Y))
T = np.array(list(T))

In [ ]:

for o in range(1,5000):
    A1,A2,A3,Xi=forward_prop(W1,W2,W3,X)
    pred=np.argmax(A3,axis=1)
    exa=np.sum(pred==Y)
    print('acc',exa/len(X))
    dW1,dW2,dW3,loss=backward_prop(A1,A2,A3,W2,W3,T,Xi,0.000003)
    print('loss ',np.sum(loss/len(X)))
    W1=W1-dW1
    W2=W2-dW2
    W3=W3-dW3